In [40]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Patch
import seaborn as sns
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache

sns.set_style('ticks')

In [41]:
drive_dir = "/Volumes/Brain2024"
cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(cache_dir=drive_dir, use_static_cache=True)
behavior_session_id = 1053960987
behavior_session = cache.get_behavior_session(behavior_session_id)

/opt/anaconda3/envs/swdb2024/lib/python3.9/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)


In [42]:
behavior_session.rewards

,volume,timestamps,auto_rewarded
0,0.005,44.87655,True
1,0.005,64.39291,True
2,0.005,80.90632,True
3,0.003,119.55534,False
4,0.003,130.28088,False
...,...,...,...
137,0.003,3522.61835,False
138,0.003,3534.59474,False
139,0.003,3566.17148,False
140,0.003,3591.60942,False


In [43]:
licks = behavior_session.licks

In [44]:
import src_local.get_metrics as get_metrics

In [45]:
licks_annot = get_metrics.annotate_licks(licks, behavior_session)

In [7]:
licks_annot[licks_annot['bout_rewarded']].head()

,timestamps,frame,pre_ili,post_ili,bout_start,bout_end,bout_number,rewarded,num_rewards,bout_rewarded,bout_num_rewards
7,45.53275,1279,6.26005,0.13111,True,False,6,True,1,True,1
8,45.66386,1287,0.13111,0.17113,False,False,6,False,0,True,1
9,45.83499,1297,0.17113,0.11008,False,False,6,False,0,True,1
10,45.94507,1304,0.11008,0.17815,False,False,6,False,0,True,1
11,46.12322,1314,0.17815,0.15011,False,False,6,False,0,True,1


In [46]:
stimulus_presentations = behavior_session.stimulus_presentations
stimulus_presentations = stimulus_presentations[stimulus_presentations['active']]
stim_table = get_metrics.annotate_bouts(licks_annot, stimulus_presentations)

In [47]:
stim_table.columns

Index(['stimulus_block', 'image_name', 'duration', 'start_time', 'end_time',
       'start_frame', 'end_frame', 'is_change', 'is_image_novel', 'omitted',
       'flashes_since_change', 'trials_id', 'active', 'spatial_frequency',
       'rewarded', 'position_y', 'stimulus_name', 'stimulus_index',
       'orientation', 'temporal_frequency', 'position_x', 'color',
       'is_sham_change', 'contrast', 'bout_start', 'num_bout_start',
       'bout_number', 'bout_end', 'num_bout_end', 'in_lick_bout', 'licked'],
      dtype='object')

In [50]:
stim_table['hits']   =  \
    stim_table['licked'] & \
    stim_table['is_change']
stim_table['misses'] = \
    ~stim_table['licked'] & \
    stim_table['is_change']
stim_table['aborts'] =  \
    stim_table['licked'] & \
    ~stim_table['is_change']

In [51]:
columns = ['start_time','hits','misses',
    'aborts','is_change','omitted','licked','bout_start',
    'bout_end','num_bout_start','num_bout_end','in_lick_bout']
df = pd.DataFrame(data = stim_table[columns])
df = df.rename(columns={'is_change':'change'})

In [52]:
import psytrack as psy

In [53]:
help(psy.hyperOpt)

Help on function hyperOpt in module psytrack.hyperOpt:

hyperOpt(dat, hyper, weights, optList, method=None, showOpt=0, jump=2, hess_calc='weights')
    Optimizes for hyperparameters and weights.
    
    Given data and set of hyperparameters, uses decoupled Laplace to find the
    optimal hyperparameter values (i.e. the sigmas that maximize evidence).
    
    Args:
        dat : dict, all data from a specific subject
        hyper : dict, hyperparameters and initial values used to construct prior
            Must at least include sigma, can also include sigInit, sigDay
        weights : dict, name and count of which weights in dat['inputs'] to fit
            Note that since dicts have no ordering, the weights returned as an
            array in `best_wMode` will be ordered alphabetically.
        optList : list, hyperparameters in 'hyper' to be optimized
        method : str, control over type of fit,
            None is standard, '_days' and '_constant' also supported
        showOp

In [56]:
import src.psy_tools as ps

In [57]:
format_options = {'timing0/1':True, 
                  'mean_center':True, 
                  'timing_params': [4,-5],
                  'ignore_trial_errors': False,
                  'num_cv_folds': 10}

In [58]:
# Define columns of interest from the stimulus_presentations DataFrame
columns = ['start_time','hits','misses',
    'aborts','is_change','omitted','licked','bout_start',
    'bout_end','num_bout_start','num_bout_end','in_lick_bout']

# Create a new DataFrame with only the selected columns
df = pd.DataFrame(data = stim_table[columns])

# Rename the 'is_change' column to 'change' for clarity
df = df.rename(columns={'is_change':'change'})

# Process behavior annotations

# Create 'y' column: 2 if bout_start, 1 otherwise (target variable for prediction)
df['y'] = np.array([2 if x else 1 for x in 
    stim_table.bout_start.values]) # predict bout start

# Calculate the **number** of images since the last lick
df['images_since_last_lick'] = stim_table.groupby(\
    stim_table['bout_end'].cumsum()).cumcount(ascending=True)

# Create 'timing_input': shift 'images_since_last_lick' by 1 and add 1 to each value
df['timing_input'] = [x+1 for x in df['images_since_last_lick'].shift(fill_value=0)]


# Build Strategy regressors

# Create various encodings for the 'change' variable
df['task0']      = np.array([1 if x else 0 for x in df['change']]) 
# Create encodings for omissions
df['omissions']  = np.array([1 if x else 0 for x in df['omitted']]) # omission
df['omissions1'] = np.array([x for x in np.concatenate([[0], # shift omission 
                    df['omissions'].values[0:-1]])]) 

# Build timing strategy using average timing parameters
df['timing1D']          = np.array(\
    [ps.timing_sigmoid(x,format_options['timing_params']) 
    for x in df['timing_input']])

# Add a bias term (constant feature)
df['bias'] = 1

In [62]:
# Create 'included' column: True if not in a lick bout
df['included'] = ~df['in_lick_bout']

In [64]:
df.shape

(4804, 21)

In [59]:
import copy 

In [65]:
# Make a copy before trimming data
full_df = copy.copy(df) 
# Segment out consumption trials
df = df[df['included']] # this excusion is becaouse the 'in_lick_bout' state has only to do with consumption
# Numer of trails that we leave out:
df['missing_trials'] = np.concatenate([np.diff(df.index)-1,[0]])


/var/folders/45/vxs3qxtn0ql1mmj2dm1w3zxw0000gp/T/ipykernel_63321/350689509.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing_trials'] = np.concatenate([np.diff(df.index)-1,[0]])
